# TensorFlow Recommenders: Quickstart

install and import TFRS

In [5]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [6]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

read the data

In [7]:
ratings = tfds.load('movielens/100k-ratings', split='train')
movies = tfds.load('movielens/100k-movies', split='train')

ratings = ratings.map(lambda x: {
    'movie_title': x['movie_title'],
    'user_id': x['user_id']
})
movies = movies.map(lambda x: x['movie_title'])

2022-06-22 21:07:07.751862: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteYYXFR5/movielens-train.tfrecord*...:   0…

Dataset movielens downloaded and prepared to ~/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [9]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x['user_id']))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

2022-06-22 21:09:08.219817: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


define a model

In [10]:
class MovieLensModel(tfrs.Model):
    def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
        super().__init__()
        
        self.user_model = user_model
        self.movie_model = movie_model
        
        self.task = task
        
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features['user_id'])
        movie_embeddings = self.movie_model(features['movie_title'])
        
        return self.task(user_embeddings, movie_embeddings)

In [12]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
        movies.batch(128).map(movie_model)
    )
)

fit and evaluate it

In [13]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(ratings.batch(4096), epochs=3)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

_, titles = index(np.array(['42']))
print(f'Top 3 recommendations for user 42: {titles[0, :3]}')

Epoch 1/3
25/25 [==============================] - 14s 495ms/step - factorized_top_k/top_1_categorical_accuracy: 1.3000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0021 - factorized_top_k/top_10_categorical_accuracy: 0.0056 - factorized_top_k/top_50_categorical_accuracy: 0.0496 - factorized_top_k/top_100_categorical_accuracy: 0.1110 - loss: 33085.1011 - regularization_loss: 0.0000e+00 - total_loss: 33085.1011
Epoch 2/3
25/25 [==============================] - 15s 626ms/step - factorized_top_k/top_1_categorical_accuracy: 3.7000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0057 - factorized_top_k/top_10_categorical_accuracy: 0.0154 - factorized_top_k/top_50_categorical_accuracy: 0.1075 - factorized_top_k/top_100_categorical_accuracy: 0.2129 - loss: 31008.9416 - regularization_loss: 0.0000e+00 - total_loss: 31008.9416
Epoch 3/3
25/25 [==============================] - 12s 474ms/step - factorized_top_k/top_1_categorical_accuracy: 7.5000e-04 - factorized_top_k/top_5_categor